<a href="https://colab.research.google.com/github/SorrowAddict/SK-FLY-AI/blob/main/ChatBot/240201_%EC%8B%A4%EC%8A%B503_%EB%A7%90%EB%AD%89%EC%B9%98_CNN%EA%B0%90%EC%A0%95%EB%B6%84%EB%A5%98%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 말뭉치 -> CNN 감정 분류 모델 실습

In [ ]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

In [26]:
import pandas as pd

# JSON 파일 경로
train_file = "/content/data/1-1.여성의류(196).json"

# JSON 파일을 읽어옵니다.
data = pd.read_json(train_file)

# "Aspects" 열에서 "SentimentText"를 추출하여 리스트로 변환합니다.
features = []
for aspects in data['Aspects']:
    for aspect in aspects:
        features.append(aspect['SentimentText'])

# "Aspects" 열에서 "SentimentPolarity"를 추출하여 리스트로 변환합니다.
labels = []
label_mapping = {-1: 2}
for aspects in data['Aspects']:
    for aspect in aspects:
        original_label = int(aspect["SentimentPolarity"])
        modified_label = label_mapping.get(original_label, original_label)
        labels.append(modified_label)


In [27]:
# 단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
word_index = tokenizer.word_index
MAX_SEQ_LEN = 15  # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [28]:
# 학습용, 검증용, 테스트용 데이터셋 생성 ➌
# 학습셋:검증셋:테스트셋 = 7:2:1
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)
train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

In [29]:
# 하이퍼파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(word_index) + 1  # 전체 단어 수

In [30]:
# CNN 모델 정의
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(filters=128, kernel_size=3, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)
conv2 = Conv1D(filters=128, kernel_size=4, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)
conv3 = Conv1D(filters=128, kernel_size=5, padding='valid', activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

# 3, 4, 5- gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])
hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(3, name='logits')(dropout_hidden)
predictions = Dense(3, activation=tf.nn.softmax)(logits)

In [31]:
# 모델 생성
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [32]:
# 모델 학습
model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

Epoch 1/5
33/33 [==============================] - 9s 82ms/step - loss: 0.4900 - accuracy: 0.8768 - val_loss: 0.4758 - val_accuracy: 0.8743
Epoch 2/5
33/33 [==============================] - 1s 45ms/step - loss: 0.4539 - accuracy: 0.8799 - val_loss: 0.4626 - val_accuracy: 0.8689
Epoch 3/5
33/33 [==============================] - 1s 27ms/step - loss: 0.4077 - accuracy: 0.8846 - val_loss: 0.3894 - val_accuracy: 0.8689
Epoch 4/5
33/33 [==============================] - 1s 18ms/step - loss: 0.3692 - accuracy: 0.8799 - val_loss: 0.2693 - val_accuracy: 0.8852
Epoch 5/5
33/33 [==============================] - 0s 14ms/step - loss: 0.2524 - accuracy: 0.8783 - val_loss: 0.1993 - val_accuracy: 0.8907


In [33]:
# 모델 평가(테스트 데이터셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

5/5 [==============================] - 0s 58ms/step - loss: 0.1126 - accuracy: 0.9451
Accuracy: 94.505495
loss: 0.112612


In [34]:
# 모델 저장
model.save('cnn_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [35]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing

In [36]:
# 감정 분류 CNN 모델 불러오기
model = load_model('cnn_model.h5')
model.summary()
model.evaluate(test_ds, verbose=2)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 15)]                 0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 15, 128)              320640    ['input_2[0][0]']             
                                                                                                  
 dropout_2 (Dropout)         (None, 15, 128)              0         ['embedding_1[0][0]']         
                                                                                                  
 conv1d_3 (Conv1D)           (None, 13, 128)              49280     ['dropout_2[0][0]']           
                                                                                            

[0.17102578282356262, 0.8901098966598511]

In [37]:
# 테스트용 데이터셋의 100번째 데이터 출력
print("단어 시퀀스 : ", corpus[100])
print("단어 인덱스 시퀀스 : ", padded_seqs[100])
print("문장 분류(정답) : ", labels[100])

단어 시퀀스 :  ['요즘날씨에도', '하나만', '입기', '딱좋겠어요']
단어 인덱스 시퀀스 :  [961 962  32 963   0   0   0   0   0   0   0   0   0   0   0]
문장 분류(정답) :  1


In [40]:
# 테스트용 데이터셋의 100번째 데이터 감정 예측
picks = [100]
predict = model.predict(padded_seqs[picks])
predict_class = tf.math.argmax(predict, axis=1)
print("여성 의류 리뷰 예측 점수 : ", predict)
print("여성 의류 리뷰 예측 클래스 : ", predict_class.numpy(), "     중립:0, 긍정:1, 부정:2")

1/1 [==============================] - 0s 20ms/step
여성 의류 리뷰 예측 점수 :  [[0.04623234 0.859559   0.09420863]]
여성 의류 리뷰 예측 클래스 :  [1]      중립:0, 긍정:1, 부정:2
